# Pylops-distributed - Solvers

In this notebook we investigate the use of scipy solvers with distributed operators. 
We will also create a simple CG solver to compare the results and performance

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import skfmm
import dask.array as da
import pylops
import pylops_distributed

from scipy.sparse.linalg.interface import MatrixLinearOperator, aslinearoperator 
from scipy.sparse import csr_matrix, vstack
from scipy.linalg import lstsq, solve
from scipy.sparse.linalg import LinearOperator, cg, lsqr, gmres
#
from scipy.signal import convolve, filtfilt

In [2]:
client = pylops_distributed.utils.backend.dask()
client

Client Scheduler: inproc://10.0.0.15/2801/1 Dashboard: http://localhost:8787/status,Cluster Workers: 1 Cores: 1 Memory: 8.59 GB


Let's just try out the solver with a matrix 

In [3]:
n = 10
x = np.ones(n)

A = np.random.randn(n, n)
A = np.dot(A.T, A)
print('eigs', np.linalg.eig(A)[0])

#A = MatrixLinearOperator(A)
Aop = aslinearoperator(A)

y = Aop.matvec(x)
xinv_sp = cg(Aop, y, maxiter=n)[0]
xinv = pylops_distributed.optimization.cg.cg(Aop, y, np.zeros_like(x), n)
print(xinv_sp)
print(xinv)

eigs [33.65686494 21.07652073 18.02672081 14.20569459  2.80215499  0.09772934
  1.76915307  0.60114319  1.04114368  1.47055489]
[0.99999997 0.99999997 1.00000004 0.99999997 1.00000001 1.
 0.99999994 1.00000002 0.99999998 0.99999997]
[0.99999993 0.99999995 1.00000008 0.99999994 1.00000002 0.99999999
 0.99999988 1.00000005 0.99999996 0.99999994]


And with the LinearOperator

In [4]:
Ada = da.from_array(A, chunks=(n//2, n//2))
Aop = pylops_distributed.MatrixMult(Ada, compute=(True, True))
yy = Aop * da.ones(n, chunks=(n//2,))

xinv_sp = cg(Aop, yy, maxiter=n)[0]
xinv = pylops_distributed.optimization.cg.cg(Aop, yy, da.zeros(n), n)
print(xinv_sp)
print(xinv)
print(xinv.compute())

[0.9999999  0.99999992 1.00000012 0.99999991 1.00000003 0.99999999
 0.99999983 1.00000007 0.99999994 0.99999992]
dask.array<add, shape=(10,), dtype=float64, chunksize=(10,)>
[0.9999999  0.99999992 1.00000011 0.99999991 1.00000003 0.99999999
 0.99999983 1.00000007 0.99999994 0.99999992]


In [5]:
%timeit -n 3 -r 1 cg(Aop, y, maxiter=n)[0]
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cg(Aop, y, da.zeros(n), n).compute()
%timeit -n 3 -r 1 pylops_distributed.optimization.cg.cg(Aop, y, da.zeros(n), n, compute=True)

603 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
1.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
1.02 s ± 0 ns per loop (mean ± std. dev. of 1 run, 3 loops each)
